# Apuntes

In [1]:
# Librerías:
import pandas as pd             # Manejo de datos
import numpy as np              # Manejo de arrays n-dimensionales y func. matemáticas
import scipy                    # Manejo de func. matemáticas y distrib. de probabilidad
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, MaxAbsScaler, StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import random                   # Generación aleatoria
import matplotlib.pyplot as plt # Generación de gráficos

In [2]:
# Base de datos de prueba
from sklearn.datasets import load_iris

In [3]:
### Abrir la Base de Datos: ###

# bd = pd.read_csv('C:/Users/Diego/OneDrive - Universidad Rey Juan Carlos/Documentos/GIA_URJC/Curso 2023-24/G.-IA/G.-IA/Curso_2/Cuatri_2/AprendizajeAutomatico_1/Apuntes/california_housing_apuntes.csv', sep=',')

# Base de datos de prueba:
iris_data = load_iris()
bd = pd.DataFrame(iris_data.data, columns=iris_data.feature_names)
bd['species'] = pd.Categorical.from_codes(iris_data.target, iris_data.target_names)

# Guardar tamaño de bd:
N, D = bd.shape

In [4]:
### Dividir en Train-Test: ###

fraction_test = 0.2

# Crea una lista con los índices de la bd:
ind = bd.index.tolist()
# Desordena los índices:
random.shuffle(ind)

# Calcula la cantidad de ejemplos que se guardan en test:
N_test = int(N * fraction_test)

# Divide los datos:
test_df = bd.iloc[ind[:N_test]]
train_df = bd.iloc[ind[N_test:]]


In [5]:
# Imprime el tamaño de las bd:
print(f'BD size: ({N}, {D})\nTrain size: {train_df.shape} \t Test size: {test_df.shape}')

BD size: (150, 5)
Train size: (120, 5) 	 Test size: (30, 5)


In [6]:
# Guardar los datos de train y Test en un directorio:

# flag_save = True
# if flag_save:
#     train_folder = 'C:/Users/Diego/OneDrive - Universidad Rey Juan Carlos/Documentos/GIA_URJC/Curso 2023-24/G.-IA/G.-IA/Curso_2/Cuatri_2/AprendizajeAutomatico_1/Apuntes/'
#     train_name   = 'train_df_apuntes.csv'
#     train_df.to_csv(train_folder + train_name, sep=';', header=True)

# if flag_save:
#     test_folder = 'C:/Users/Diego/OneDrive - Universidad Rey Juan Carlos/Documentos/GIA_URJC/Curso 2023-24/G.-IA/G.-IA/Curso_2/Cuatri_2/AprendizajeAutomatico_1/Apuntes/'
#     test_name   = 'test_apuntes.csv'
#     test_df.to_csv(test_folder + test_name, sep=';', header=True)

In [7]:
# Información de los datos para ver si hay que codificar:
train_df.info()

### Tipos de datos: Atributos 'continuos', 'discretos' y 'categóricos' ###

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 34 to 30
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   sepal length (cm)  120 non-null    float64 
 1   sepal width (cm)   120 non-null    float64 
 2   petal length (cm)  120 non-null    float64 
 3   petal width (cm)   120 non-null    float64 
 4   species            120 non-null    category
dtypes: category(1), float64(4)
memory usage: 4.9 KB


In [8]:
### La codificación de los tipos categóricos no siempre hay que hacerla por la falta de estos###
# 1) Averiguamos las columnas categóricas:
cat_cols = train_df.select_dtypes(include='category').columns.tolist()

# 2) Creamos un dataframe con las columnas categóricas:
train_cod = train_df

# 3) Codificamos la fila, a la vez que creamos una diccionario de diccionarios para descodificar en el futuro
dict_decode={}
for col in cat_cols:
  codes = train_df[col].cat.codes                   # Codifica la serie del df actual
  code_to_categ = dict(zip(codes,train_df[col]))    # Crea la relación de codificación
  train_cod[col] = codes                            # Asigna el valor codificado a la serie
  dict_decode[col] = code_to_categ                  # Añade la decodificación al diccionario

print(f'Columnas categóricas: {cat_cols}\n{train_cod.head()}\nDecodificación: {dict_decode}')

Columnas categóricas: ['species']
     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
17                 5.1               3.5                1.4               0.3   
23                 5.1               3.3                1.7               0.5   
103                6.3               2.9                5.6               1.8   
72                 6.3               2.5                4.9               1.5   
110                6.5               3.2                5.1               2.0   

     species  
17         0  
23         0  
103        2  
72         1  
110        2  
Decodificación: {'species': {0: 'setosa', 2: 'virginica', 1: 'versicolor'}}


C:\Users\Diego\AppData\Local\Temp\ipykernel_13412\1781162713.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_cod[col] = codes                            # Asigna el valor codificado a la serie


In [9]:
### Tambien se puede codificar por One-Hot. Elegir cómo nos viene mejor. ###
### Codificación One-hot: crea una columna por cada categoría validando con 1 y 0 ###
OneHot_codification = train_df
for col in cat_cols:
    OneHot_codification = pd.concat([OneHot_codification, pd.get_dummies(OneHot_codification[col])], axis=1)

OneHot_codification

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species,0,1,2
17,5.1,3.5,1.4,0.3,0,1,0,0
23,5.1,3.3,1.7,0.5,0,1,0,0
103,6.3,2.9,5.6,1.8,2,0,0,1
72,6.3,2.5,4.9,1.5,1,0,1,0
110,6.5,3.2,5.1,2.0,2,0,0,1
...,...,...,...,...,...,...,...,...
137,6.4,3.1,5.5,1.8,2,0,0,1
124,6.7,3.3,5.7,2.1,2,0,0,1
13,4.3,3.0,1.1,0.1,0,1,0,0
34,4.9,3.1,1.5,0.2,0,1,0,0


In [10]:
# Ver la información del df
print(train_cod.info(), '\n')        # Indica el tamaño, los tipos de datos y cuántos son 'Na' de cada atributo
print(train_cod.describe(), '\n')    # Indica la descripción estadística básica excepto la moda de cada atributo en forma de tabla
train_cod.mode(axis=0, dropna=False) # Devuelve una tabla con las modas de cada atributo

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 17 to 119
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  120 non-null    float64
 1   sepal width (cm)   120 non-null    float64
 2   petal length (cm)  120 non-null    float64
 3   petal width (cm)   120 non-null    float64
 4   species            120 non-null    int8   
dtypes: float64(4), int8(1)
memory usage: 4.8 KB
None 

       sepal length (cm)  sepal width (cm)  petal length (cm)  \
count         120.000000        120.000000         120.000000   
mean            5.845833          3.004167           3.826667   
std             0.822263          0.411269           1.728249   
min             4.300000          2.000000           1.000000   
25%             5.100000          2.775000           1.600000   
50%             5.800000          3.000000           4.400000   
75%             6.400000          3.225000      

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.0,3.0,1.5,0.2,2.0
1,6.3,NaN,NaN,NaN,NaN


### Búsqueda de valores NaN:

In [11]:
# Búsqueda de valores NaN:
missing_data = train_cod.isna()
missing_data.sum()

missing_values_per_column = missing_data.sum(axis=0)    # 'NA' por cada columna
missing_values_per_row = missing_data.sum(axis=1)       # 'NA' por cada fila

mask_mayorq0 = missing_values_per_column > 0            # Crea una máscara de Pandas para indicar si hay columnas con NA
mask_mayorq1 = missing_values_per_row > 0               # Crea una máscara de Pandas para indicar si hay filas con NA

print(f'Columnas con valores nulos:\n{missing_values_per_column[mask_mayorq0]}\n')
print(f'Filas con valores nulos:\n{missing_values_per_row[mask_mayorq1]}\n')

missing_count_row = missing_values_per_row.value_counts().sort_index()
print(f'Valores NaN en cada fila:\n{missing_count_row}')
missing_count_col = missing_values_per_column.value_counts().sort_index()
print(f'Valores NaN en cada columna:\n{missing_count_col}')

Columnas con valores nulos:
Series([], dtype: int64)

Filas con valores nulos:
Series([], dtype: int64)

Valores NaN en cada fila:
0    120
dtype: int64
Valores NaN en cada columna:
0    5
dtype: int64


##### Se puede hacer una de las siguientes cosas con los valores NaN:
- Eliminar filas/columnas Nan
- Imputación univariada
- Imputación multivariante

In [ ]:
### Eliminar filas y columnas con NaN excesivos ###

# Eliminar filas:
if missing_count_row.index[-1] > 0:                                         # Calcular la cantidad máxima de valores nulos por fila
    mask_toDrop = missing_values_per_row >= missing_count_row.index[-1]     # Filtro que busca las filas con el número de valores perdidos máximo
    drop_list_row = missing_values_per_row[mask_toDrop].index.tolist()      # Crea una lista de índices de las filas que cumplen con la condición
    train_cod.drop(drop_list_row, inplace=True)                             # Eliminar las filas guardadas en 'drop_list' del DataFrame original

# Eliminar columnas:                                      
if missing_count_col.index[-1] > 0:                                     # Calcular la cantidad máxima de valores nulos por columna
    mask_toDrop = missing_values_per_column >= missing_count_col.index[-1]  # Filtro que busca las filas con el número de valores perdidos máximo
    drop_list_col = missing_values_per_column[mask_toDrop].index.tolist()   # Crea una lista de índices de las columnas que cumplen con la condición
    train_cod.drop(drop_list_col, inplace=True)                             # Eliminar las columnas guardadas en 'drop_list' del DataFrame original, al usar implace no hace falta poner 'df = ...'

# Tamaño del DataFrame redimensionado:
train_cod.shape

In [ ]:
### Imputación univariada de los datos NaN: ###

train_cod.fillna(train_cod.median(axis=0), inplace=True)    # Sustituye los valores NaN por el valore de la mediana
train_cod.shape
# Se pueden sustituir por la media o el valor más repetido (moda)
# El problema de esta técnica es que puede crear ejemplos imposibles como decir que un hombre está embarazado

In [ ]:
### Imputación multivariante de los datos NaN: ###

imputer = IterativeImputer()
train_imputed = imputer.fit_transform(train_cod)

# Convertir de nuevo a DataFrame
train_imputed = pd.DataFrame(train_imputed, columns=train_cod.columns)

# Recuento de NaN:
train_imputed.isna().sum()

### Ingeniería de características:

In [13]:
# Aumento de la dimensionalidad de la tabla:
train_dim = train_cod
degree = 2
interaction_only = True

polyf = PolynomialFeatures(degree=degree, interaction_only=interaction_only, include_bias=False)    # include_bias es una columna de sesgo (todo 1)
polyf.set_output(transform="pandas")

# No se puede hacer esta línea de comando con el test NUNCA:
polyf.fit(train_dim)
# Adapta los datos a las nuevas características:
train_dim = polyf.transform(train_dim)

In [43]:
print(f'Info. Dataframe aumentado:\n')
train_dim.info()

Info. Dataframe aumentado:

<class 'pandas.core.frame.DataFrame'>
Int64Index: 120 entries, 17 to 119
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   sepal length (cm)                    120 non-null    float64
 1   sepal width (cm)                     120 non-null    float64
 2   petal length (cm)                    120 non-null    float64
 3   petal width (cm)                     120 non-null    float64
 4   species                              120 non-null    float64
 5   sepal length (cm) sepal width (cm)   120 non-null    float64
 6   sepal length (cm) petal length (cm)  120 non-null    float64
 7   sepal length (cm) petal width (cm)   120 non-null    float64
 8   sepal length (cm) species            120 non-null    float64
 9   sepal width (cm) petal length (cm)   120 non-null    float64
 10  sepal width (cm) petal width (cm)    120 non-null    float64
 11  sep

In [34]:
### Transformaciones sin modificar la dimensionalidad ###

# Escalado al intervalo unidad:
train_MinMaxScaler = train_dim

scaler = MinMaxScaler()
scaler.fit(train_MinMaxScaler)
train_MinMaxScaler = scaler.transform(train_MinMaxScaler)

# Esclalado al máximo en valor absoluto:
train_MaxAbScaler = train_dim

scaler = MaxAbsScaler()
scaler.fit(train_MaxAbScaler)
train_MaxAbScaler = scaler.transform(train_MaxAbScaler)

# Estandarización:
train_Standar = train_dim

scaler = StandardScaler()
scaler.fit(train_Standar)
train_Standar = scaler.transform(train_Standar)

In [ ]:
# Dibujo de la tabla de correlación de los diferentes atributos:

corr_matrix = train_dim.corr()

fig, ax = plt.subplots(figsize=(10,8))          # Tamaño de la figura
cax = ax.matshow(corr_matrix, cmap='coolwarm')  # Dibujar el mapa de calor

fig.colorbar(cax, label='Correlación')
plt.title('Mapa de calor de la matriz de correlación')
ax.set_yticks(np.arange(len(corr_matrix.columns)))
ax.set_yticklabels(corr_matrix.columns)

plt.show()